In [1]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

import py_scripts.db_fcns as db
import py_scripts.data_fcns as fcns

In [2]:
alkuasema = "JÄS"
loppuasema = "OV"
# dates = pd.date_range("2023-01-01", periods=2, freq="1D")
dates = pd.date_range("2022-04-01", "2022-09-30", freq="1D")
# date = "2023-"

timetables = pd.DataFrame()
for date in dates:
    new_table = fcns.get_train_nums(alkuasema, loppuasema, str(date.date()))
    if new_table is None:
        continue
    new_table["departureDate"] = str(date.date())
    timetables = pd.concat([timetables, new_table])
# train_nums = fcns.get_train_nums(alkuasema, loppuasema, date)

In [3]:
timetables["scheduledTime"] = pd.to_datetime(timetables["scheduledTime"])

In [4]:
timetables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182220 entries, 24 to 37
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype              
---  ------            --------------   -----              
 0   stationShortCode  182220 non-null  object             
 1   stationUICCode    182220 non-null  int64              
 2   countryCode       182220 non-null  object             
 3   type              182220 non-null  object             
 4   trainStopping     182220 non-null  bool               
 5   commercialStop    34664 non-null   object             
 6   commercialTrack   182220 non-null  object             
 7   cancelled         182220 non-null  bool               
 8   scheduledTime     182220 non-null  datetime64[ns, UTC]
 9   trainNumber       182220 non-null  int64              
 10  departureDate     182220 non-null  object             
dtypes: bool(2), datetime64[ns, UTC](1), int64(2), object(6)
memory usage: 14.2+ MB


In [5]:
timetables["type"].unique()

array(['DEPARTURE', 'ARRIVAL'], dtype=object)

In [6]:
timetables.head(10)

,stationShortCode,stationUICCode,countryCode,type,trainStopping,commercialStop,commercialTrack,cancelled,scheduledTime,trainNumber,departureDate
24,TPE,160,FI,DEPARTURE,True,True,3,False,2022-04-01 04:04:00+00:00,80,2022-04-01
25,SJ,157,FI,ARRIVAL,False,NaN,,False,2022-04-01 04:10:00+00:00,80,2022-04-01
26,SJ,157,FI,DEPARTURE,False,NaN,,False,2022-04-01 04:10:00+00:00,80,2022-04-01
27,VTR,1295,FI,ARRIVAL,False,NaN,,False,2022-04-01 04:12:00+00:00,80,2022-04-01
28,VTR,1295,FI,DEPARTURE,False,NaN,,False,2022-04-01 04:12:00+00:00,80,2022-04-01
29,LPÄ,156,FI,ARRIVAL,True,True,1,False,2022-04-01 04:15:00+00:00,80,2022-04-01
30,LPÄ,156,FI,DEPARTURE,True,True,1,False,2022-04-01 04:16:00+00:00,80,2022-04-01
31,MAT,1172,FI,ARRIVAL,False,NaN,,False,2022-04-01 04:19:00+00:00,80,2022-04-01
32,MAT,1172,FI,DEPARTURE,False,NaN,,False,2022-04-01 04:19:00+00:00,80,2022-04-01
33,VIA,155,FI,ARRIVAL,False,NaN,,False,2022-04-01 04:21:00+00:00,80,2022-04-01


In [7]:
uniques = timetables.apply(lambda r: (r["trainNumber"], r["departureDate"]), axis=1).unique()

In [8]:
len(uniques)

1684

In [9]:
# time_df = pd.DataFrame()
# for num, date in uniques:
    # time_df = pd.concat([time_df, fcns.get_train_timetable(num, date)])

In [10]:
# time_df.info()

In [11]:
# time_df["differenceInMinutes"].describe()

In [12]:
# df[df["station"] == alkuasema].index.max()

In [13]:
# num = df.iloc[1157, :]["trainNumber"]
# date = df.iloc[1157, :]["departureDate"]
# d = df.iloc[1157, :]["dist_from_speed"]
# c = df.iloc[1157, :]["dist_from_coords"]

In [14]:
def some_dict(train_num, date, d, c):
    return {"trainNumber": train_num, 
            "departureDate": date, 
            "dist_from_speed": d, 
            "dist_from_coords": c
           }

In [15]:
# some_dict(num, date, d, c)

In [16]:
# test = pd.DataFrame()
# test = pd.concat([test, pd.DataFrame([some_dict(num, date, d, c)])])
# test = pd.concat([test, pd.DataFrame([some_dict(num, date, d, c)])])
# test

In [17]:
def get_data():
    df = pd.DataFrame()
    distances = pd.DataFrame()

    for num, date in uniques:
        try:
            new_df = fcns.get_train_location_data(num, date, False)
        except KeyError as e:
            print(f"{type(e)}: {e} [{date=}, {num=}]")
            continue
        if new_df is None:
            continue
            
        alku = new_df[new_df["station"] == alkuasema]
        if len(alku) > 0:
            index1 = alku.index.max()
        else:
            continue
        loppu = new_df[new_df["station"] == loppuasema]
        if len(loppu) > 0:
            index2 = loppu.index.min()
        else:
            continue
        dist_from_speed = new_df.loc[index2, "dist_from_speed"] - new_df.loc[index1, "dist_from_speed"]
        dist_from_coords = new_df.loc[index2, "dist_from_coords"] - new_df.loc[index1, "dist_from_coords"]
        distances = pd.concat([distances, pd.DataFrame([some_dict(num, date, dist_from_speed, dist_from_coords)])])

        new_df = new_df.loc[index1:index2, :]
        new_df["dist_from_speed"] = new_df["dist_from_speed"] - new_df["dist_from_speed"].min()
        new_df["dist_from_coords"] = new_df["dist_from_coords"] - new_df["dist_from_coords"].min()
        df = pd.concat([df, new_df])
        
        time.sleep(0.3)
        
    return [df, distances]

In [18]:
df = None
distances = None

use_db = False

if use_db:
    df = db.get_df_from_db("testi")
    distances = db.get_df_from_db("testi_distances")

if df is None or distances is None:
    df, distances = get_data()

Data not found: train_num=148, date='2022-04-02'
Data not found: train_num=80, date='2022-04-04'
Data not found: train_num=148, date='2022-04-24'
Data not found: train_num=148, date='2022-05-07'
Data not found: train_num=148, date='2022-05-12'
Data not found: train_num=148, date='2022-05-19'
Data not found: train_num=148, date='2022-06-01'
Data not found: train_num=80, date='2022-06-27'
Data not found: train_num=80, date='2022-07-01'
Data not found: train_num=94, date='2022-08-19'
Data not found: train_num=140, date='2022-09-19'


In [19]:
# 42766 entries, 13 columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161180 entries, 122 to 548
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   trainNumber         161180 non-null  int64              
 1   departureDate       161180 non-null  object             
 2   timestamp           161180 non-null  datetime64[ns, UTC]
 3   speed               161180 non-null  int64              
 4   duration            161180 non-null  float64            
 5   latitude            161180 non-null  float64            
 6   longitude           161180 non-null  float64            
 7   acceleration        161180 non-null  float64            
 8   change_of_location  161180 non-null  float64            
 9   dist_from_coords    161180 non-null  float64            
 10  dist_from_speed     161180 non-null  float64            
 11  stops_from_speed    161180 non-null  int64              
 12  station          

In [20]:
# 212 entries, 4 columns
distances.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1305 entries, 0 to 0
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   trainNumber       1305 non-null   int64  
 1   departureDate     1305 non-null   object 
 2   dist_from_speed   1305 non-null   float64
 3   dist_from_coords  1305 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 51.0+ KB


In [21]:
distances.loc[:, ["dist_from_speed", "dist_from_coords"]].describe()

,dist_from_speed,dist_from_coords
count,1305.000000,1305.000000
mean,55878.060026,56699.175514
std,954.979897,9089.027928
min,40549.166667,51220.111309
25%,55786.111111,55438.758124
50%,55901.388889,55528.163342
75%,56000.833333,55735.921545
max,62320.555556,356808.125073


In [22]:
# mean / median

# koko tammikuu
# based on speed
# 55.75 km / 55.90 km
# based on coords
# 55.25 km / 55.61 km

# 2 ekaa päivää
# based on speed
# 55.97 km / 55.91 km
# based on coords
# 55.34 km / 55.55 km

In [23]:
best_dist = round(distances["dist_from_speed"].mean(), -2)
best_dist

55900.0

In [24]:
# joku yritys
# df.groupby(["trainNumber", "departureDate"])[["dist_from_coords", "dist_from_speed"]].describe()

In [25]:
# joitain kiihtyvyyksiä puuttuu?
df[df["acceleration"].isna()]

,trainNumber,departureDate,timestamp,speed,duration,latitude,longitude,acceleration,change_of_location,dist_from_coords,dist_from_speed,stops_from_speed,station


In [26]:
# "testi"
# import py_scripts.db_fcns as db
# db.save_df_to_db(df, "testi")

In [27]:
# db.save_df_to_db(distances, "testi_distances")

In [28]:
distances.reset_index(drop=True, inplace=True)

In [29]:
trains = []
for i in distances.index:
    trains.append((distances.loc[i, "trainNumber"], distances.loc[i, "departureDate"]))

In [30]:
len(trains)

1305

In [31]:
main_df = df.loc[:, ["trainNumber", "departureDate", "dist_from_speed", "acceleration"]].copy()
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161180 entries, 122 to 548
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   trainNumber      161180 non-null  int64  
 1   departureDate    161180 non-null  object 
 2   dist_from_speed  161180 non-null  float64
 3   acceleration     161180 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 6.1+ MB


In [32]:
main_df.reset_index(drop=True, inplace=True)

In [33]:
# fcns.get_locations_for_train(88, "2023-01-01", main_df).tail()

In [34]:
# main_df[(main_df["trainNumber"] == 88) & (main_df["departureDate"] == "2023-01-01") & (main_df["dist_from_speed"] == 55700)]

In [35]:
step_length = 100
checkpoints = np.arange(0, best_dist + 1, step_length)

In [36]:
for num, date in trains:
    d = main_df[(main_df["trainNumber"] == num) & (main_df["departureDate"] == date)]["dist_from_speed"]
    i1 = d.index.min()
    i2 = d.index.max()
    main_df.loc[i1:i2, "dist_from_speed"] = d * best_dist / d.max()

In [37]:
def accel_dict(num, date, dist, accel=None):
    return {"trainNumber": num,
            "departureDate": date,
            "dist_from_speed": dist,
            "acceleration": accel
           }

In [38]:
#for num, date in trains:
    #if (maksimi := main_df[(main_df["trainNumber"] == num) & (main_df["departureDate"] == date)]["dist_from_speed"].max()) != best_dist:
        # print(f"ropleema: {maksimi} != {best_dist}")

In [39]:
additions = []
for num, date in trains:
    for d in checkpoints[1:-1]:
        additions.append(accel_dict(num, date, d))

In [40]:
main_df = pd.concat([main_df, pd.DataFrame(additions)])

In [41]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889370 entries, 0 to 728189
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   trainNumber      889370 non-null  int64  
 1   departureDate    889370 non-null  object 
 2   dist_from_speed  889370 non-null  float64
 3   acceleration     161180 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 33.9+ MB


In [42]:
main_df = main_df.sort_values(["departureDate", "trainNumber", "dist_from_speed"])

In [43]:
main_df = main_df.fillna(method="ffill")

In [44]:
main_df.head(20)

,trainNumber,departureDate,dist_from_speed,acceleration
507,86,2022-04-01,0.000000,0.000000
508,86,2022-04-01,0.000000,0.277778
509,86,2022-04-01,62.419974,0.388889
2232,86,2022-04-01,100.000000,0.388889
2233,86,2022-04-01,200.000000,0.388889
510,86,2022-04-01,212.227913,0.407407
2234,86,2022-04-01,300.000000,0.407407
2235,86,2022-04-01,400.000000,0.407407
511,86,2022-04-01,453.585147,0.296296
2236,86,2022-04-01,500.000000,0.296296


In [45]:
trains[0]

(140, '2022-04-01')

In [46]:
# lienee turha
def peculiar_rounding(num, reference=best_dist):
    res = round(num)
    if res == reference:
        return reference
    return res

In [47]:
main_df["dist_from_speed"] = main_df["dist_from_speed"].apply(round)

In [48]:
# main_df[(main_df["trainNumber"] == 88) & (main_df["departureDate"] == "2023-01-01")].tail(1)["dist_from_speed"].to_numpy()[0]

In [49]:
# onko ongelmia?
for num, date in trains:
    d = checkpoints[-1]
    if len(main_df[(main_df["trainNumber"] == num) & (main_df["departureDate"] == date) & (main_df["dist_from_speed"] == d)]) == 0:
        print(num, date, d)

In [50]:
main_df.head(2)

,trainNumber,departureDate,dist_from_speed,acceleration
507,86,2022-04-01,0,0.000000
508,86,2022-04-01,0,0.277778


In [51]:
ongelma = fcns.get_locations_for_train(88, "2021-06-04", main_df)

In [52]:
len(ongelma)

0

In [53]:
ongelma.head(2)

,trainNumber,departureDate,dist_from_speed,acceleration


In [54]:
ongelma.tail(2)

,trainNumber,departureDate,dist_from_speed,acceleration


In [55]:
main_df.tail(2)

,trainNumber,departureDate,dist_from_speed,acceleration
724283,150,2022-09-30,55800,-0.500000
160241,150,2022-09-30,55900,-0.611111


In [56]:
testi = main_df[main_df["dist_from_speed"].isin(checkpoints)].copy()

In [57]:
len(testi)

733679

In [58]:
testi.drop_duplicates(["departureDate", "trainNumber", "dist_from_speed"], inplace=True)

In [59]:
len(testi)

730800

In [60]:
for num, date in trains:
    if len(testi[(testi["trainNumber"] == num) & (testi["departureDate"] == date)]) != len(checkpoints):
        print(num, date)

In [61]:
testi.reset_index(drop=True, inplace=True)

In [62]:
testi["acceleration+"] = testi["acceleration"].apply(lambda a: max(a, 0))

In [63]:
testi["acceleration_abs"] = testi["acceleration"].apply(abs)

In [64]:
testi.head()

,trainNumber,departureDate,dist_from_speed,acceleration,acceleration+,acceleration_abs
0,86,2022-04-01,0,0.000000,0.000000,0.000000
1,86,2022-04-01,100,0.388889,0.388889,0.388889
2,86,2022-04-01,200,0.388889,0.388889,0.388889
3,86,2022-04-01,300,0.407407,0.407407,0.407407
4,86,2022-04-01,400,0.407407,0.407407,0.407407


In [65]:
# MUISTA

# pd.pivot_table(testi, values="acceleration", index=["departureDate", "trainNumber"], columns=["dist_from_speed"], aggfunc=np.mean)

In [66]:
def get_cluster_df(df, col_name="acceleration"):
    result = pd.pivot_table(df, values=col_name, index=["departureDate", "trainNumber"], columns=["dist_from_speed"], aggfunc=np.mean)
    return result.dropna()

In [67]:
clustering_df = get_cluster_df(testi)

In [68]:
# clustering_df.dropna(inplace=True)

In [69]:
def get_clustering_df(nums_and_dates, df, clustering_columns, col_name="acceleration"):
    clustering_df = pd.DataFrame(columns=clustering_columns)
    
    for num, date in nums_and_dates:
        partial_df = fcns.get_locations_for_train(num, date, df)
        i1 = partial_df.index.min()
        i2 = partial_df.index.max()
        accel = df.loc[i1:i2, col_name].copy().to_numpy()
        clustering_df = pd.concat([clustering_df, pd.DataFrame([accel], columns=clustering_columns)])
        
    clustering_df.index = nums_and_dates
    
    return clustering_df

In [70]:
#clustering_df = pd.DataFrame(columns=checkpoints)

#for num, date in trains:
#    dataa = fcns.get_locations_for_train(num, date, testi)
#    i1 = dataa.index.min()
#    i2 = dataa.index.max()
#    accel = testi.loc[i1:i2, "acceleration"].copy().to_numpy()
#    clustering_df = pd.concat([clustering_df, pd.DataFrame([accel], columns=checkpoints)])

In [71]:
# clustering_df.index = trains

In [72]:
clustering_df.insert(0, "mean_accel", clustering_df.mean(axis=1))

In [73]:
# clustering_df.head()

In [74]:
len(clustering_df)

1305

In [75]:
# from sklearn.cluster import KMeans

In [76]:
# km = KMeans(n_clusters=15, n_init="auto", max_iter=1337, random_state=4096)
km = KMeans(n_clusters=25, n_init="auto")
if "cluster_id" in clustering_df.columns:
    clustering_df.drop("cluster_id", axis=1, inplace=True)
km.fit(clustering_df.drop("mean_accel", axis=1))

if "cluster_id" in clustering_df.columns:
    clustering_df.drop("cluster_id", axis=1, inplace=True)
clustering_df.insert(0, "cluster_id", km.predict(clustering_df.drop("mean_accel", axis=1)))

In [77]:
clusters = clustering_df.groupby("cluster_id")

table = pd.concat([clusters["mean_accel"].count(), clusters["mean_accel"].min(), clusters["mean_accel"].max()], axis=1)
table.columns=["count", "min mean accel", "max mean accel"]
clustering_results = table.sort_values("count", ascending=False)
clustering_results.head()

,count,min mean accel,max mean accel
cluster_id,,,
12,670,-0.086573,0.159353
0,605,-0.173025,0.102336
5,3,-0.354890,-0.054341
3,3,-0.226239,-0.105710
9,2,0.171762,0.296593


In [78]:
# clustering_df.head()

In [79]:
# cluster_df_pos = get_clustering_df(trains, testi, checkpoints, "acceleration+")
cluster_df_pos = get_cluster_df(testi, "acceleration+")

In [80]:
len(cluster_df_pos)

1305

In [81]:
km_pos = KMeans(n_clusters=25, n_init="auto")

if "mean_accel+" in cluster_df_pos.columns:
    cluster_df_pos.drop("mean_accel+", axis=1, inplace=True)
if "cluster_id" in cluster_df_pos.columns:
    cluster_df_pos.drop("cluster_id", axis=1, inplace=True)
km_pos.fit(cluster_df_pos)
cluster_ids = km_pos.predict(cluster_df_pos)

cluster_df_pos.insert(0, "mean_accel+", cluster_df_pos.mean(axis=1))
cluster_df_pos.insert(0, "cluster_id", cluster_ids)

In [82]:
clusters_pos = cluster_df_pos.groupby("cluster_id")

table = pd.concat([clusters_pos["mean_accel+"].count(), clusters_pos["mean_accel+"].min(), clusters_pos["mean_accel+"].max()], axis=1)
table.columns=["count", "min mean accel+", "max mean accel+"]
c_counts = table.sort_values("count", ascending=False)
c_counts.head(8)

,count,min mean accel+,max mean accel+
cluster_id,,,
19,615,0.014517,0.084476
5,580,0.012068,0.105170
0,43,0.030026,0.219358
4,26,0.040249,0.161036
1,9,0.107893,0.359815
8,4,0.058323,0.109213
13,4,0.062358,0.107211
15,4,0.047082,0.082274


In [83]:
# cluster_df_pos.iloc[:6, :6]

In [84]:
def print_stuff(cluster_id):
    a = clustering_df[clustering_df["cluster_id"] == cluster_id]
    print(a.iloc[0, :2])
    
    plt.plot(a.iloc[0, 2:])
    plt.grid()
    plt.show()
    print()
    

In [85]:
# for i in table.index:
    # if table.loc[i, "count"] == 1:
        # print_stuff(i)

In [86]:
c_df_res = clustering_df.loc[:, ["cluster_id", "mean_accel"]]
c_df = clustering_df.drop(["cluster_id", "mean_accel"], axis=1)
# c_df_res = cluster_df_pos.loc[:, ["cluster_id", "mean_accel+"]]
# c_df = cluster_df_pos.drop(["cluster_id", "mean_accel+"], axis=1)

In [87]:
default_lower_bound = max(30, 0.05 * len(clustering_df))
# default_lower_bound = 0.1 * len(cluster_df_pos)

def get_replacements(cluster_counts, lower_bound=default_lower_bound):
    clusters = cluster_counts[cluster_counts > lower_bound]
    return {c: i for i, c in enumerate(clusters.index)}

In [88]:
get_replacements(clustering_results["count"])
# get_replacements(c_counts["count"])

{12: 0, 0: 1}

In [89]:
c_df_res.groupby("cluster_id").count().sort_values("mean_accel", ascending=False).head()
# c_df_res.groupby("cluster_id").count().sort_values("mean_accel+", ascending=False).head()

dist_from_speed,mean_accel
cluster_id,
12,670
0,605
5,3
3,3
9,2


In [90]:
def replacement_fcn(num, cluster_counts):
    replacements = get_replacements(cluster_counts)
    # replacements = {22: 0, 19: 1, 23: 2, 17: 3, 24: 4}
    if replacements.get(num) is None:
        return len(replacements)
    return replacements[num]

In [91]:
c_df_res["cluster_id"] = c_df_res["cluster_id"].apply(lambda n: replacement_fcn(n, clustering_results["count"]))
# c_df_res["cluster_id"] = c_df_res["cluster_id"].apply(lambda n: replacement_fcn(n, c_counts["count"]))

In [92]:
c_df_res.groupby("cluster_id").count().sort_values("mean_accel", ascending=False)
# c_df_res.groupby("cluster_id").count().sort_values("mean_accel+", ascending=False)

dist_from_speed,mean_accel
cluster_id,
0,670
1,605
2,30


In [93]:
X_train, X_test, Y_train, Y_test = train_test_split(c_df, c_df_res["cluster_id"].to_numpy(), test_size=0.2)

In [94]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)

In [95]:
accuracy_score(Y_test, Y_pred)

0.8237547892720306

In [96]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83       133
           1       0.81      0.82      0.82       125
           2       1.00      0.67      0.80         3

    accuracy                           0.82       261
   macro avg       0.88      0.77      0.82       261
weighted avg       0.82      0.82      0.82       261

